In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from hilbertcurve.hilbertcurve import HilbertCurve

### Constants

In [ ]:
LANE_CHANGE_DATA_20HZ_DIR = './data/lane_changes/20hz/'
LANE_CHANGE_DATA_10HZ_DIR = './data/lane_changes/10hz/'
LANE_CHANGE_DATA_5HZ_DIR = './data/lane_changes/5hz/'
LANE_CHANGE_HZ_DIRS = [LANE_CHANGE_DATA_20HZ_DIR, LANE_CHANGE_DATA_10HZ_DIR, LANE_CHANGE_DATA_5HZ_DIR]
OFFSET = 10
FACTOR_MULTIPLY = 10000
SAMPLING_RATE_VARIATIONS = 3 # constant to denote the amount of different sampling rates that exist

### Dataframes

In [ ]:
laneChange20HzDFs = [] # Will contain the dataframes for all lane changes.
laneChange10HzDFs = [] # Will contain the dataframes for all lane changes.
laneChange5HzDFs = [] # Will contain the dataframes for all lane changes.

# Count number of files (= amt of lane changes)
numOfFiles = len([f for f in os.listdir(LANE_CHANGE_DATA_20HZ_DIR) if os.path.isfile(os.path.join(LANE_CHANGE_DATA_20HZ_DIR, f))])

# Import each csv into a dataframe
for i in range(numOfFiles):
    temp_20_df = pd.read_csv(LANE_CHANGE_DATA_20HZ_DIR + "lane_change_" + str(i+1) + ".csv", sep=";")
    temp_10_df = pd.read_csv(LANE_CHANGE_DATA_10HZ_DIR + "lane_change_" + str(i+1) + ".csv", sep=";")
    temp_5_df = pd.read_csv(LANE_CHANGE_DATA_5HZ_DIR + "lane_change_" + str(i+1) + ".csv", sep=";")
    laneChange20HzDFs.append(temp_20_df)
    laneChange10HzDFs.append(temp_10_df)
    laneChange5HzDFs.append(temp_5_df)

# Append all arrays to a single array so that
# future data processing can be done more efficiently.
laneChangeDfs = [laneChange20HzDFs, laneChange10HzDFs, laneChange5HzDFs]

### Apply constants

In [ ]:
for laneChanges in laneChangeDfs:
    for df in laneChanges:
        df['accel_lon'] = df['accel_lon'].add(OFFSET)
        df['accel_trans'] = df['accel_trans'].add(OFFSET)
        df['accel_down'] = df['accel_down'].add(OFFSET)

        df['accel_lon'] = df.apply(lambda x: int(x['accel_lon']*FACTOR_MULTIPLY) , axis=1)
        df['accel_trans'] = df.apply(lambda x: int(x['accel_trans']*FACTOR_MULTIPLY) , axis=1)
        df['accel_down'] = df.apply(lambda x: int(x['accel_down']*FACTOR_MULTIPLY) , axis=1)

### Hilbert indexing

In [ ]:
resolution = 18; dimension = 2

for laneChanges in laneChangeDfs:
    for df in laneChanges:
        points = df[['accel_lon', 'accel_trans']].to_numpy()
        # Declare Hilbert curve
        hilbert_curve = HilbertCurve(resolution, dimension, n_procs=-1)
        # Generate Hilbert index
        distances = hilbert_curve.distances_from_points(points)
        # Add Hilbert index to 'hilbert' column
        df['hilbert'] = pd.DataFrame(distances, columns = ['hilbert'] )

### Hilbert Plots

In [ ]:
lcCounter = 1 # used to track which lane change the CSP corresponds to.

print("FIRST SET IS 20HZ, SECOND SET IS 10HZ, THIRD SET IS 5 HZ ")

for laneChanges in laneChangeDfs:
    lcCounter = 1
    print("================================================================")
    print("================================================================")
    for df in laneChanges:
        print("Lane change file - " + str(lcCounter) + ": ")

        plt.hist(df['hilbert'], bins=200)
        plt.ylim(0,1)
        plt.show()

        print("-----------------------------")

        lcCounter += 1